In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import matplotlib.pyplot as plt
import pandas as pd
from src.model import BasicModel

plt.rcParams['figure.figsize'] = 15.7,13.27
plt.rcParams['font.family'] = 'AppleGothic'

class LEARN_INFO:
    def __init__(self, name, columns, is_contain_cluster_label=False, matching_type="general", jump=3):
        self.name = name
        self.columns = columns
        self.is_label = is_contain_cluster_label
        self.matching = matching_type
        self.jump = jump
    

# INPUT_REPORT_SIMULATION = [
#     LEARN_INFO(name="univariate", columns=['energy (kw 15min)'], is_contain_cluster_label=False),
#     LEARN_INFO(name="with cluster label", columns=['energy (kw 15min)'], is_contain_cluster_label=True),
# ]

DEFAULT_COLUMNS = [
    'energy (kw 15min)',
    ['week sin',
    'week cos'],
    ['year sin',
    'year cos'],
    'season idx',
    'weather idx',
    'avg ta',
    'avg rhm'
]

In [8]:
_ = {
    "univariate": ['energy (kw 15min)'],
    "multivariate (week)": [
        'energy (kw 15min)',
        'week sin',
        'week cos'
    ],
    "multivariate (year)": [
        'energy (kw 15min)',
        'year sin',
        'year cos',
    ],
    "multivariate (weather)": [
        'energy (kw 15min)',
        'weather idx'
    ],
    "multivariate (avg ta)": [
        'energy (kw 15min)',
        'avg ta'
    ],
    "multivariate (avg rhm)": [
        'energy (kw 15min)',
        'avg rhm'
    ],
    "multivariate (season)": [
        'energy (kw 15min)',
        'season idx'
    ]
}

# Input Report Simulation
IRS = []

for key in _.keys():
    IRS.append(
        LEARN_INFO(name=key, 
                   columns=_[key], 
                   is_contain_cluster_label=False),
    )

for key in _.keys():
    IRS.append(
        LEARN_INFO(name="{} / with cluster".format(key), 
                   columns=_[key], 
                   is_contain_cluster_label=True),
    )

# Training Model Array 
TMA = []

is_switch = True
test_evaluate_df = pd.DataFrame();
val_evaluate_df = pd.DataFrame();

for _ in IRS:
    print("--- now info ---")

    bm = BasicModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch,
                   matching_type=_.matching,
                   is_multi_step=True,
                   model_type="Multi Step / SingleShot")
    print(test_evaluate_df)
    print(val_evaluate_df)
    print()
    bm.set_window(OUT_STEPS=3)
    bm.set_model()
    print(bm)
    bm.training(epochs=50)
    
    bm.set_multi_predict(is_reshape=True)
    bm.set_multi_predict(is_reshape=True, is_val_datas=True)
    
    TMA.append(bm)
    
    test_evaluate_df = test_evaluate_df.append(bm.statistic(), ignore_index=True)
    val_evaluate_df = val_evaluate_df.append(bm.statistic(is_val_datas=True), ignore_index=True)
    
test_evaluate_df.index = [_.name for _ in TMA]
val_evaluate_df.index = [_.name for _ in TMA]

###### [Notice] (multivariate (season) / with cluster) set predict (validation) info start ###### 

0 / 171
50 / 171
100 / 171
150 / 171
171 / 171 complete.

###### [Notice] set predict info success ###### 



In [6]:
test_evaluate_df

,cos,ecv,mae,map,mean dis,mean sim,mse,tss,wss
univariate,0.883833,60.919466,0.278586,0.012556,1.691003,0.876514,0.158554,1456.910039,569.368229
multivariate (week),0.893040,63.605173,0.272509,0.012275,1.620493,0.884872,0.147658,1456.910039,530.239892
multivariate (year),0.826210,43.474503,0.370082,0.008771,2.085683,0.790535,0.229330,1456.910039,823.525640
multivariate (weather),0.884437,61.115935,0.277840,0.014577,1.686386,0.877150,0.157757,1456.910039,566.505846
multivariate (avg ta),0.837154,44.538007,0.333822,0.019643,1.995280,0.840338,0.225016,1456.910039,808.031342
multivariate (avg rhm),0.878066,59.252198,0.288552,0.015698,1.730076,0.871499,0.165319,1456.910039,593.658813
multivariate (season),0.872302,57.592850,0.289376,0.008806,1.756409,0.864947,0.172051,1456.910039,617.834025
univariate / with cluster,0.888333,59.495180,0.289680,0.010146,1.701863,0.886808,0.164333,1456.910039,590.118784
multivariate (week) / with cluster,0.898775,62.975474,0.279806,0.010200,1.628977,0.890813,0.150213,1456.910039,539.414036
multivariate (year) / with cluster,0.855245,50.707709,0.339062,0.010316,1.959841,0.828453,0.199985,1456.910039,718.144339


In [7]:
val_evaluate_df

,cos,ecv,mae,map,mean dis,mean sim,mse,tss,wss
univariate,0.907050,66.011890,0.266257,0.026911,1.664790,0.886918,0.148063,832.494756,282.949234
multivariate (week),0.909485,66.852338,0.270444,0.028001,1.624627,0.888557,0.144402,832.494756,275.952544
multivariate (year),0.811984,34.674087,0.436292,0.017505,2.377205,0.765484,0.284581,832.494756,543.834798
multivariate (weather),0.907023,65.996857,0.263414,0.029070,1.661208,0.886124,0.148129,832.494756,283.074380
multivariate (avg ta),0.900863,63.240897,0.261754,0.026292,1.706916,0.871812,0.160135,832.494756,306.017607
multivariate (avg rhm),0.905476,65.416688,0.272807,0.020914,1.684051,0.884327,0.150656,832.494756,287.904255
multivariate (season),0.901193,63.747734,0.272005,0.025675,1.722454,0.877464,0.157927,832.494756,301.798212
univariate / with cluster,0.935497,75.008120,0.242098,0.029990,1.437722,0.916116,0.108873,832.494756,208.056094
multivariate (week) / with cluster,0.929024,73.396167,0.250441,0.042673,1.462941,0.909235,0.115895,832.494756,221.475516
multivariate (year) / with cluster,0.869575,52.416337,0.356385,0.008896,2.020592,0.834355,0.207290,832.494756,396.131499


In [9]:
test_evaluate_df

,cos,ecv,mae,map,mean dis,mean sim,mse,tss,wss
univariate,0.906828,65.933511,0.266802,0.024517,1.666385,0.887279,0.148405,832.494756,283.601733
multivariate (week),0.913528,68.090674,0.261360,0.013587,1.593190,0.890993,0.139008,832.494756,265.643468
multivariate (year),0.867346,52.502974,0.345751,0.033787,2.013938,0.830135,0.206913,832.494756,395.410252
multivariate (weather),0.908661,66.572008,0.260825,0.019164,1.651533,0.887346,0.145623,832.494756,278.286280
multivariate (avg ta),0.891611,60.169193,0.275782,0.035730,1.778235,0.859155,0.173516,832.494756,331.589378
multivariate (avg rhm),0.904363,64.964912,0.276094,0.017583,1.698947,0.883869,0.152624,832.494756,291.665271
multivariate (season),0.896060,61.801444,0.281469,0.039353,1.776550,0.868946,0.166406,832.494756,318.000978
univariate / with cluster,0.928407,71.858892,0.256800,0.027824,1.524113,0.908327,0.122592,832.494756,234.273251
multivariate (week) / with cluster,0.929197,72.346154,0.254868,0.020128,1.497999,0.908849,0.120469,832.494756,230.216818
multivariate (year) / with cluster,0.900353,60.734656,0.331274,0.013240,1.841822,0.874641,0.171053,832.494756,326.881930


In [10]:
val_evaluate_df

,cos,ecv,mae,map,mean dis,mean sim,mse,tss,wss
univariate,0.884618,61.064567,0.277744,0.019711,1.687865,0.877287,0.157966,1456.910039,567.254232
multivariate (week),0.897957,64.900677,0.267698,0.018095,1.583890,0.889655,0.142402,1456.910039,511.365567
multivariate (year),0.865212,54.907277,0.303645,0.011176,1.816277,0.855920,0.182946,1456.910039,656.960415
multivariate (weather),0.884703,61.171029,0.276285,0.016028,1.685851,0.877274,0.157534,1456.910039,565.703184
multivariate (avg ta),0.869339,56.542781,0.293277,0.022436,1.754463,0.870178,0.176311,1456.910039,633.132590
multivariate (avg rhm),0.877575,59.081373,0.286930,0.010819,1.731596,0.870983,0.166012,1456.910039,596.147584
multivariate (season),0.862579,54.407111,0.303596,0.011397,1.830900,0.852846,0.184976,1456.910039,664.247379
univariate / with cluster,0.899266,61.163557,0.289941,0.011379,1.677024,0.894638,0.157564,1456.910039,565.812041
multivariate (week) / with cluster,0.905116,63.222837,0.281765,0.019020,1.616814,0.899419,0.149209,1456.910039,535.810176
multivariate (year) / with cluster,0.888520,56.831718,0.325365,0.003183,1.830853,0.872101,0.175139,1456.910039,628.923038
